In [1]:
import numpy as np
import nltk # type: ignore 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize # type: ignore
from nltk.corpus import stopwords # type: ignore
import string
import numpy as np
from preprocess import extract_features, build_freq, word_freq
from sklearn.preprocessing import MinMaxScaler


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\obinw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\obinw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\obinw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\obinw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
file_path_positive = '../data/positive_tweets.txt'
file_path_negative = '../data/negative_tweets.txt'
import math
pos_freqs = build_freq(file_path_positive)  # Build frequency dictionary
# print("The psoitive frequency")
# print(pos_freqs)
neg_freqs = build_freq(file_path_negative)
# print("The negative frquency")
# print(neg_freqs)
word_f = word_freq(pos_freqs, neg_freqs, "../data/sample_sentiment_text.txt")
#print("The frequency of the sentences ares ", word_f)
matrix = extract_features(word_f)
scaler = MinMaxScaler()
matrix = scaler.fit_transform(matrix)

print(matrix)

[[0.         0.625      0.33333333]
 [0.         0.25       0.46666667]
 [0.         0.375      0.06666667]
 [0.         0.4375     0.        ]
 [0.         0.125      0.46666667]
 [0.         0.0625     0.13333333]
 [0.         0.125      0.13333333]
 [0.         0.0625     0.        ]
 [0.         0.1875     0.2       ]
 [0.         0.0625     0.06666667]
 [0.         0.1875     0.2       ]
 [0.         0.         0.        ]
 [0.         0.125      0.13333333]
 [0.         0.875      0.        ]
 [0.         0.125      0.33333333]
 [0.         0.         0.46666667]
 [0.         0.9375     0.66666667]
 [0.         0.         0.26666667]
 [0.         0.5625     0.13333333]
 [0.         0.25       0.93333333]
 [0.         0.875      0.2       ]
 [0.         0.5625     1.        ]
 [0.         0.875      0.        ]
 [0.         0.5        0.93333333]
 [0.         0.375      0.        ]
 [0.         0.1875     0.66666667]
 [0.         0.625      0.6       ]
 [0.         0.25       0.66

In [3]:
import pandas as pd
#initialize some parameters
parameter = np.array([0.005, 0.0001, 0.0001])
b = 0.05
df = pd.read_csv("../data/output.txt")
m = len(df)
target = df.values
print(target)

[[1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]]


In [4]:
#I am building my classification model
#the logixtic model
def model(matrix, parameter, b):
    y_hat = []
    for row in matrix:
        feature = row
        z = np.dot(feature, parameter) + b
        g =  1 / (1 + np.exp(-z))
        y_hat.append(g)
    return np.array(y_hat)

In [5]:
#getting my cost and loss functions
import math
#compute the loss function
def loss_function(target, y_hat):
    cost = 0
    for x in range(m):
        output = target[x,0]
        loss = (-output * math.log(y_hat[x])) - ((1 - output) * math.log(1 - y_hat[x]))
        cost += loss
    cost =  cost/m
    return cost

In [18]:
def gradient_descent(matrix, target, parameter, b, rate, iterations):
    print("Beginning Parameters:", parameter)
    print("beginning Bias:", b)
    m = len(target)
    for _ in range(iterations):
        y_hat = model(matrix, parameter, b)
        d_parameter = np.zeros_like(parameter)
        d_b = 0
        for i in range(m):
            error = (target[i]- y_hat[i])
            d_parameter += error * matrix[i]
            d_b += error
        parameter += (rate / m) * d_parameter
        b -= (rate / m) * d_b
        
        if _ % 100 == 0:
            cost = loss_function(target, y_hat)
            print(f"Iteration {_}, Cost: {cost}")
    
    return parameter, b
rate = 0.001
iterations = 1000

parameter, b = gradient_descent(matrix, target, parameter, b, rate, iterations)
print("Updated Parameters:", parameter)
print("Updated Bias:", b)
#feature increased so you have to adjust the parameters

Beginning Parameters: [ 0.005       0.07612883 -0.06773525]
beginning Bias: [-0.04217752]
Iteration 0, Cost: 0.6868688945618382
Iteration 100, Cost: 0.6869043059106783
Iteration 200, Cost: 0.6869910961628841


C:\Users\obinw\AppData\Local\Temp\ipykernel_8212\3223544191.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  loss = (-output * math.log(y_hat[x])) - ((1 - output) * math.log(1 - y_hat[x]))


Iteration 300, Cost: 0.6871311568054101
Iteration 400, Cost: 0.6873264531791123
Iteration 500, Cost: 0.6875790268672173
Iteration 600, Cost: 0.6878909981107081
Iteration 700, Cost: 0.6882645682450553
Iteration 800, Cost: 0.6887020221519861
Iteration 900, Cost: 0.6892057307191684
Updated Parameters: [ 0.005       0.15799021 -0.12671959]
Updated Bias: [-0.15638033]


In [24]:
#test data
decision_boundary = 0.5
#Now the model has been trained, now it is time to test the model and see how accyrate it is. 
parameter = [ 0.005, 0.07612883, -0.06773525]
b= [-0.04217752]
word_f = word_freq(pos_freqs, neg_freqs, "../output/test_sample.txt")
#print("The frequency of the sentences are ", word_f)
matrix = extract_features(word_f)
#print("The matric is ", matrix)
y_hat = model(matrix, parameter, b)
print("The prediction of the statements are: ", y_hat)

The prediction of the statements are:  [[0.70606911]
 [0.49070669]
 [0.54959041]
 [0.39288639]
 [0.56643922]
 [0.47798284]
 [0.62342656]
 [0.51183401]
 [0.63919066]
 [0.4401919 ]
 [0.5870686 ]
 [0.49280451]
 [0.56643922]
 [0.43385236]
 [0.4443327 ]
 [0.56643922]
 [0.54543154]
 [0.4443327 ]
 [0.62735937]
 [0.4401919 ]]


In [36]:
decision_boundary = 0.5
with open("../output/predicted.txt", "w") as file:
    for row in y_hat:
        if row[0] > decision_boundary:
            file.write("tweet is a nice tweet" + "\n")
            print("tweet is a nice tweet")
        else:
            print("Tweet is a negative tweet.")
            file.write("tweet is a negative tweet"+ "\n")

tweet is a nice tweet
Tweet is a negative tweet.
tweet is a nice tweet
Tweet is a negative tweet.
tweet is a nice tweet
Tweet is a negative tweet.
tweet is a nice tweet
tweet is a nice tweet
tweet is a nice tweet
Tweet is a negative tweet.
tweet is a nice tweet
Tweet is a negative tweet.
tweet is a nice tweet
Tweet is a negative tweet.
Tweet is a negative tweet.
tweet is a nice tweet
tweet is a nice tweet
Tweet is a negative tweet.
tweet is a nice tweet
Tweet is a negative tweet.
